In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import pandas as pd
import unicodedata
import re
import numpy as np
import os
from os import path
import io
import time
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, GRU,LSTM, Multiply
from keras.layers import RepeatVector,TimeDistributed, Dense, Activation, Lambda,BatchNormalization
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K


2022-08-16 16:08:54.759906: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-16 16:08:54.759972: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
%%bash
wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz
mkdir /tmp/data/
gunzip -c download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz > /tmp/lines
split -a 3 -l 100000  /tmp/lines /tmp/data/lines-

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
os.listdir('/tmp/data/')

['lines-cww',
 'lines-eje',
 'lines-gag',
 'lines-gis',
 'lines-drt',
 'lines-cwo',
 'lines-euw',
 'lines-fra',
 'lines-bkk',
 'lines-fsz',
 'lines-fdk',
 'lines-dot',
 'lines-czy',
 'lines-efb',
 'lines-fxo',
 'lines-fyl',
 'lines-chm',
 'lines-brc',
 'lines-bxu',
 'lines-esl',
 'lines-eab',
 'lines-bhr',
 'lines-eeu',
 'lines-fhm',
 'lines-fgr',
 'lines-aff',
 'lines-bnp',
 'lines-bir',
 'lines-css',
 'lines-elp',
 'lines-dzk',
 'lines-fhy',
 'lines-ars',
 'lines-cot',
 'lines-bgk',
 'lines-bam',
 'lines-bbr',
 'lines-dsy',
 'lines-cfi',
 'lines-bwn',
 'lines-fhh',
 'lines-cbo',
 'lines-bah',
 'lines-bwj',
 'lines-dib',
 'lines-ffc',
 'lines-fel',
 'lines-gmd',
 'lines-fwi',
 'lines-gdz',
 'lines-eza',
 'lines-ckt',
 'lines-cfp',
 'lines-gco',
 'lines-fpi',
 'lines-cwr',
 'lines-asb',
 'lines-dgd',
 'lines-ali',
 'lines-eli',
 'lines-gkb',
 'lines-eyg',
 'lines-gck',
 'lines-ckw',
 'lines-dzi',
 'lines-cds',
 'lines-frl',
 'lines-eda',
 'lines-ger',
 'lines-crh',
 'lines-acy',
 'line

In [8]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()
  w = '<start> ' + w + ' <end>'
  return w  

In [9]:
def line_cleaner(line):
  #line =""+str(line)
  line=line.decode("utf-8")
  #print(type(line))
  x=line.strip(" -.!?\n")
  return x

In [10]:
lines=[]
i=0
f= open('/tmp/data/lines-fli','rb')
for line in f:
  #print(line)
  line=line_cleaner(line)
  i=i+1
  if(i>=10000):
    break
  lines.append(line)
print(i,lines)

10000 ['Okay, see you tonight', 'Yeah', 'Tonight', 'Hey, guys', 'Come on in', 'Good to see you', "Oh, I'm so excited to have the kids", "Listen, I don't want you to worry", 'I', "I've read all the latest childcare periodicals, and I'm CPR certified, and I've stocked up on organic snacks and some really fun learning games for the kids", 'So, if you have any questions, feel free', 'Just one', 'Do you have the numbers for the local authorities', 'Oh, yeah, of course', 'Yeah', 'Super', "I'll see you later", 'Okay', 'Hey, kids', 'What do you wanna do first', 'Kids', "I'd say nice spread, but that's probably inappropriate for a teacher", "Bet you wouldn't have a problem saying it to Brooke", "Aren't you her teacher, too", 'I get it', 'You like to be the center of attention, get a rise out of people', "It's the hallmark of insecurity", 'Wow', "You're an underpaid high school teacher, sleeping with a student, and I'm insecure", 'Yeah, okay', "You seem to be in the driver's seat, Miss Gatina", 

In [12]:
train_data, validation_data = train_test_split(lines, test_size=.1, random_state=1234)
print(len(train_data))

Tx=20
vocab_size=10000
padding='post'
trunc_type='post'
oov_tok='<OOV>'


tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(lines)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(lines)
padded=pad_sequences(sequences,maxlen=Tx,padding=padding,truncating=trunc_type)
print(padded[:10])

8999
[[  40   71    2  107    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [  52    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [ 107    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [  53  123    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [  64   18   11    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [ 108    5   71    2    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [  39   15   28  558    5   26    3  141    0    0    0    0    0    0
     0    0    0    0    0    0]
 [ 215    4   21   46    2    5  369    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   4    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [ 176  512   38    3  586  962  963    7   15  96

In [13]:
# #MODEL 2 tokenizer
# # Here I will use Tokenizer to extract the keyword vector as baseline
# # I will use train data to fit the Tokenizer, then use this Tokenizer to extract the validation data
# max_length = 100
# max_features = 50000
# token = Tokenizer(num_words=max_features)
# token.fit_on_texts(list(np.asarray(train_data.question_text)))
# xtrain = token.texts_to_sequences(np.asarray(train_data.question_text))
# xvalidate = token.texts_to_sequences(np.asarray(validation_data.question_text))
# xtest = token.texts_to_sequences(np.asarray(test.question_text))

# # Because Tokenizer will split the sentence, for some sentence are smaller,
# # so we have to pad the missing position
# xtrain = pad_sequences(xtrain, maxlen=max_length)
# xvalidate = pad_sequences(xvalidate, maxlen=max_length)
# xtest = pad_sequences(xtest, maxlen=max_length)

# ytrain = train_data.target
# yvaliate = validation_data.target

In [14]:
print(word_index)
print(len(word_index))
reverse_word_index={}
for word,ind in word_index.items():
  reverse_word_index[ind]=word
print(reverse_word_index)

{'<OOV>': 1, 'you': 2, 'the': 3, 'i': 4, 'to': 5, 'a': 6, 'and': 7, 'of': 8, 'it': 9, 'me': 10, 'in': 11, 'that': 12, 'is': 13, 'what': 14, "i'm": 15, 'for': 16, 'this': 17, 'on': 18, 'my': 19, 'know': 20, "don't": 21, 'we': 22, 'do': 23, 'was': 24, 'be': 25, 'have': 26, 'are': 27, 'so': 28, 'not': 29, "it's": 30, 'out': 31, 'your': 32, 'with': 33, 'no': 34, 'just': 35, 'but': 36, 'get': 37, 'all': 38, 'oh': 39, 'okay': 40, 'up': 41, "you're": 42, 'here': 43, 'right': 44, 'he': 45, 'want': 46, 'like': 47, 'there': 48, 'if': 49, 'can': 50, 'go': 51, 'yeah': 52, 'hey': 53, 'back': 54, 'her': 55, 'well': 56, "that's": 57, 'one': 58, 'they': 59, 'about': 60, 'think': 61, "i'll": 62, 'now': 63, 'come': 64, 'really': 65, 'nicole': 66, 'him': 67, 'got': 68, 'look': 69, 'from': 70, 'see': 71, 'when': 72, 'take': 73, 'how': 74, 'by': 75, 'gonna': 76, 'time': 77, 'some': 78, 'our': 79, 'god': 80, 'going': 81, 'why': 82, "can't": 83, 'over': 84, 'way': 85, 'at': 86, 'make': 87, 'did': 88, 'more':

In [15]:
contexts=np.array([padded[i] for i in range(0,len(lines)-1)])
responses=np.array([padded[i] for i in range(1,len(lines))])
responses=np.expand_dims(responses,-1)
print(contexts[6])
print(responses[6])
print(contexts.shape,responses.shape)

[ 39  15  28 558   5  26   3 141   0   0   0   0   0   0   0   0   0   0
   0   0]
[[215]
 [  4]
 [ 21]
 [ 46]
 [  2]
 [  5]
 [369]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]]
(9998, 20) (9998, 20, 1)


In [16]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-06-27 06:57:17--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-27 06:57:17--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-27 06:57:18--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [17]:
from zipfile import ZipFile
file_name = "/content/glove.6B.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Unzipped glove')


Unzipped glove


In [18]:
embeddings_index = {}
f = open( 'glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Number of word vectors is %s.' % len(embeddings_index))

Number of word vectors is 400000.


In [19]:
embed_dim=100
num_words=vocab_size
embedding_matrix=np.zeros((len(word_index)+1,embed_dim))
for word,i in word_index.items():
  embedding_vector=embeddings_index.get(word)
  if(embedding_vector is not None and i <num_words):
    embedding_matrix[i]=embedding_vector

In [20]:
from keras.layers import Embedding
embedding_layer=Embedding(len(word_index)+1,embed_dim,weights=[embedding_matrix],input_length=Tx,trainable=False)

In [21]:
#from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
#from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline


In [22]:
model=Sequential()
model.add(embedding_layer)
model.add(BatchNormalization())
model.add(LSTM(units=256,return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size,activation='softmax')))

In [23]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 100)           390800    
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 100)           400       
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 256)           365568    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 10000)         2570000   
Total params: 3,326,768
Trainable params: 2,935,768
Non-trainable params: 391,000
_________________________________________________________________


In [25]:
model.fit(contexts,responses,epochs=10,batch_size=10)

Epoch 1/10
9998/9998 [==============================] - 39s 4ms/step - loss: 1.0298
Epoch 2/10
9998/9998 [==============================] - 40s 4ms/step - loss: 0.9730
Epoch 3/10
9998/9998 [==============================] - 39s 4ms/step - loss: 0.9239
Epoch 4/10
9998/9998 [==============================] - 39s 4ms/step - loss: 0.8840
Epoch 5/10
9998/9998 [==============================] - 39s 4ms/step - loss: 0.8483
Epoch 6/10
9998/9998 [==============================] - 39s 4ms/step - loss: 0.8185
Epoch 7/10
9998/9998 [==============================] - 40s 4ms/step - loss: 0.7893
Epoch 8/10
9998/9998 [==============================] - 39s 4ms/step - loss: 0.7643
Epoch 9/10
9998/9998 [==============================] - 39s 4ms/step - loss: 0.7433
Epoch 10/10
9998/9998 [==============================] - 39s 4ms/step - loss: 0.7190


In [48]:
quest=["what time will you come"]
seq=tokenizer.texts_to_sequences(quest)
pad=pad_sequences(seq,maxlen=Tx,padding=padding,truncating=trunc_type)


In [49]:
pred=model.predict(pad)
pred=np.argmax(pred,axis=-1)
print(pred,pred.shape)
pred=pred.reshape(20,)
for i in pred:
  if i==0:
    continue
  else:
    print(reverse_word_index[i])

[[ 4 35 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]] (1, 20)
i
just
know


In [52]:
quest=["its your birthday today"]
seq=tokenizer.texts_to_sequences(quest)
pad=pad_sequences(seq,maxlen=Tx,padding=padding,truncating=trunc_type)
pred=model.predict(pad)
pred=np.argmax(pred,axis=-1)
print(pred,pred.shape)
pred=pred.reshape(20,)
for i in pred:
  if i==0:
    continue
  else:
    print(reverse_word_index[i])

[[  3  12   5   0   0 108   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]] (1, 20)
the
that
to
good


In [28]:
import math
def beam_search_decoder(prediction, BW = 3):

    output_sequences = [([], 0)]
    
    for sent in prediction:
        new_sequences = []
        sent = sent.reshape(20, )
        print(sent.shape)
        for old_seq, old_score in output_sequences:
            for char_index in range(len(sent)):
                new_seq = old_seq + [char_index]
                if(sent[char_index]==0):
                  continue
                else:
                  new_score = old_score + math.log(sent[char_index])
                new_sequences.append((new_seq, new_score))
                
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:BW]
        
    return output_sequences